# Data Preparation & Exploration

In [61]:
#Loading Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [62]:
import warnings

# Suppress FutureWarning
warnings.simplefilter(action='ignore', category=FutureWarning)


In [63]:
#Loading dataset
df=pd.read_csv('diabetic_data.csv')
df.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [64]:
df.shape

(101766, 50)

 We got know that our dataset has 101766 instances and 50 features


In [65]:
#checking data types of each variable
df.dtypes

encounter_id                 int64
patient_nbr                  int64
race                        object
gender                      object
age                         object
weight                      object
admission_type_id            int64
discharge_disposition_id     int64
admission_source_id          int64
time_in_hospital             int64
payer_code                  object
medical_specialty           object
num_lab_procedures           int64
num_procedures               int64
num_medications              int64
number_outpatient            int64
number_emergency             int64
number_inpatient             int64
diag_1                      object
diag_2                      object
diag_3                      object
number_diagnoses             int64
max_glu_serum               object
A1Cresult                   object
metformin                   object
repaglinide                 object
nateglinide                 object
chlorpropamide              object
glimepiride         

### Checking for missing values
 In our dataset NULL values are represted by '?' so replacing it with NULL to get count of NULL values

In [66]:
#checking missing values
df.replace('?',np.nan,inplace=True)
df.isnull().sum()

encounter_id                    0
patient_nbr                     0
race                         2273
gender                          0
age                             0
weight                      98569
admission_type_id               0
discharge_disposition_id        0
admission_source_id             0
time_in_hospital                0
payer_code                  40256
medical_specialty           49949
num_lab_procedures              0
num_procedures                  0
num_medications                 0
number_outpatient               0
number_emergency                0
number_inpatient                0
diag_1                         21
diag_2                        358
diag_3                       1423
number_diagnoses                0
max_glu_serum               96420
A1Cresult                   84748
metformin                       0
repaglinide                     0
nateglinide                     0
chlorpropamide                  0
glimepiride                     0
acetohexamide 

### Dealing with missing values
 Variable weight contains approximate 98% of the missing values so there is no significance in filling those missing values so we decided to drop these variables. Variable Payer code and medical specialty contains approximate 40% missing values so we also dropped these variables. Variables race, diag_1, diag_2, diag_3 and gender contains very less missing values as compared to other attributes which we dropped so for these attributes we also decided to drop those where missing values contains.


In [67]:
df = df.drop(['weight','payer_code','medical_specialty'], axis = 1)

In [68]:
df = df.drop_duplicates(subset= ['patient_nbr'], keep = 'first')
df.shape

(71518, 47)

In [70]:
# Identify rows where all three diagnosis columns are NaN
drop_Idx = set(df[(df['diag_1'].isnull()) & (df['diag_2'].isnull()) & (df['diag_3'].isnull())].index)

# Add rows where any of the diagnosis columns are NaN
drop_Idx = drop_Idx.union(set(df[df['diag_1'].isnull()].index))
drop_Idx = drop_Idx.union(set(df[df['diag_2'].isnull()].index))
drop_Idx = drop_Idx.union(set(df[df['diag_3'].isnull()].index))

# Add rows where the race column is NaN
drop_Idx = drop_Idx.union(set(df[df['race'].isnull()].index))

# Add rows where discharge_disposition_id is 11
drop_Idx = drop_Idx.union(set(df[df['discharge_disposition_id'] == 11].index))

# Add rows where gender is 'Unknown/Invalid'
drop_Idx = drop_Idx.union(set(df[df['gender'] == 'Unknown/Invalid'].index))

# Ensure indices are valid
drop_Idx = drop_Idx.intersection(set(df.index))

# Keep only the rows that are not in drop_Idx
new_Idx = list(set(df.index) - drop_Idx)

# Update the DataFrame
data = df.loc[new_Idx].reset_index(drop=True)


 variables (drugs named citoglipton and examide), all records have the same value. So essentially these cannot provide any interpretive or discriminatory information for predicting readmission so we decided to drop these two variables

In [71]:
df = df.drop(['citoglipton', 'examide'], axis = 1)

### Feature Engineering


This is highly subjective, and partly depends on a knowledge of health care services, and making sense of the potential relationships between features. There are perhaps thousands of ways to try here. We tried some...

Service utilization: The data contains variables for number of inpatient (admissions), emergency room visits and outpatient visits for a given patient in the previous one year. These are (crude) measures of how much hospital/clinic services a person has used in the past year. We added these three to create a new variable called service utilization (see figure below). The idea was to see which version gives us better results. Granted, we did not apply any special weighting to the three ingredients of service utilization but we wanted to try something simple at this stage

In [72]:
df['service_utilization'] = df['number_outpatient'] + df['number_emergency'] + df['number_inpatient']
df = df.drop(['number_outpatient', 'number_emergency', 'number_inpatient'], axis=1)


Number of medication changes: The dataset contains 23 features for 23 drugs (or combos) which indicate for each of these, whether a change in that medication was made or not during the current hospital stay of patient. Medication change for diabetics upon admission has been shown by previous research to be associated with lower readmission rates. We decided to count how many changes were made in total for each patient, and declared that a new feature. The reasoning here was to both simplify the model and possibly discover a relationship with number of changes regardless of which drug was changed.

In [73]:
keys = ['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'glipizide', 
        'glyburide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'insulin', 'glyburide-metformin',
        'tolazamide', 'metformin-pioglitazone','metformin-rosiglitazone', 'glimepiride-pioglitazone', 
        'glipizide-metformin', 'troglitazone', 'tolbutamide', 'acetohexamide']
for col in keys:
    colname = str(col) + 'temp'
    df[colname] = df[col].apply(lambda x: 0 if (x == 'No' or x == 'Steady') else 1)
df['numchange'] = 0
for col in keys:
    colname = str(col) + 'temp'
    df['numchange'] = df['numchange'] + df[colname]
    del df[colname]
    
df['numchange'].value_counts()  

numchange
0    53870
1    16638
2      933
3       74
4        3
Name: count, dtype: int64

Encoding some variables: The original dataset used string values for gender, race, medication change, and each of the 23 drugs used. To better fit those variables into our model, we interpret the variables to numeric binary variables to reflect their nature. For example, we encoded the “ medication change ” feature from “No” (no change) and “Ch” (changed) into 0 and 1.

In [74]:
# Replace values in specific columns
df['change'] = df['change'].replace({'Ch': 1, 'No': 0})
df['gender'] = df['gender'].replace({'Male': 1, 'Female': 0})
df['diabetesMed'] = df['diabetesMed'].replace({'Yes': 1, 'No': 0})

# Replace values in columns listed in 'keys'
replace_map = {'No': 0, 'Steady': 1, 'Up': 1, 'Down': 1}
for col in keys:
    df[col] = df[col].replace(replace_map)


In [76]:
# Replace values in 'A1Cresult'
df['A1Cresult'] = df['A1Cresult'].replace({'>7': 1, '>8': 1, 'Norm': 0, np.nan: -99})

# Replace values in 'max_glu_serum'
df['max_glu_serum'] = df['max_glu_serum'].replace({'>200': 1, '>300': 1, 'Norm': 0, np.nan: -99})


Dealing with age: There are different ways to deal with this. The dataset only gives us age as 10 year categories, so we don’t know the exact age of each patient. The previous study on this dataset used age categories as nominal variables, but we wanted to be able to see the effect of increasing age on readmission, even if in a crude way. To do that, we assume that age of the patient on average lies at the midpoint of the age category. For example, if the patient’s age category is 20–30 years, then we assume the age = 25 years. So we converted age categories to midpoints, resulting in a numeric variable:

In [77]:
# code age intervals [0-10) - [90-100) from 1-10
for i in range(0,10):
    df['age'] = df['age'].replace('['+str(10*i)+'-'+str(10*(i+1))+')', i+1)
df['age'].value_counts()

age
8     18210
7     15960
6     12466
9     11589
5      6878
4      2699
10     1900
3      1127
2       535
1       154
Name: count, dtype: int64

Collapsing of Multiple Encounters for same patient Some patients in the dataset had more than one encounter.We could not count them as independent encounters because that bias the results towards those patients who had multiple encounters.

In [78]:
df.head().T


,0,1,2,3,4
encounter_id,2278392,149190,64410,500364,16680
patient_nbr,8222157,55629189,86047875,82442376,42519267
race,Caucasian,Caucasian,AfricanAmerican,Caucasian,Caucasian
gender,0,0,0,1,1
age,1,2,3,4,5
admission_type_id,6,1,1,1,1
discharge_disposition_id,25,1,1,1,1
admission_source_id,1,7,7,7,7
time_in_hospital,1,3,2,2,1
num_lab_procedures,41,59,11,44,51


Encoding the outcome variable: The outcome we are looking at is whether the patient gets readmitted to the hospital within 30 days or not. The variable actually has < 30, > 30 and No Readmission categories. To reduce our problem to a binary classification, we combined the readmission after 30 days and no readmission into a single category:

In [79]:
df['readmitted'].value_counts()


readmitted
NO     42985
>30    22240
<30     6293
Name: count, dtype: int64

In [80]:
df['readmitted'] = df['readmitted'].replace('>30', 0)
df['readmitted'] = df['readmitted'].replace('<30', 1)
df['readmitted'] = df['readmitted'].replace('NO', 0)

Categorization of diagnoses: The dataset contained up to three diagnoses for a given patient (primary, secondary and additional). However, each of these had 700–900 unique ICD codes and it is extremely difficult to include them in the model and interpret meaningfully. Therefore, we collapsed these diagnosis codes into 9 disease categories in an almost similar fashion to that done in the original publication using this dataset. These 9 categories include Circulatory, Respiratory, Digestive, Diabetes, Injury, Musculoskeletal, Genitourinary, Neoplasms, and Others. Although we did this for primary, secondary and additional diagnoses, we eventually decided to use only the primary diagnosis in our model. Doing this in python was slightly cumbersome because, well, we are mapping the disease codes to certain category names. Below code should demonstrate this easily.

In [87]:
# Initialize level columns
for col in ['diag_1', 'diag_2', 'diag_3']:
    df[f'level1_{col}'] = df[col]
    df[f'level2_{col}'] = df[col]

# Replace non-numeric diagnostic codes
for col in ['diag_1', 'diag_2', 'diag_3']:
    df[f'level1_{col}'] = df[f'level1_{col}'].replace({'V': 0, 'E': 0, np.nan: -1})
    df[f'level2_{col}'] = df[f'level2_{col}'].replace({'V': 0, 'E': 0, np.nan: -1})

# Convert diagnostic columns to numeric
for col in ['diag_1', 'diag_2', 'diag_3']:
    df[f'level1_{col}'] = pd.to_numeric(df[f'level1_{col}'], errors='coerce').fillna(0)
    df[f'level2_{col}'] = pd.to_numeric(df[f'level2_{col}'], errors='coerce').fillna(0)

# Define categorization function for Level1
def categorize_level1(value):
    if 390 <= value < 460 or np.floor(value) == 785:
        return 1  # Cardiac
    elif 460 <= value < 520 or np.floor(value) == 786:
        return 2  # Respiratory
    elif 520 <= value < 580 or np.floor(value) == 787:
        return 3  # Digestive
    elif np.floor(value) == 250:
        return 4  # Diabetes
    elif 800 <= value < 1000:
        return 5  # Injury
    elif 710 <= value < 740:
        return 6  # Musculoskeletal
    elif 580 <= value < 630 or np.floor(value) == 788:
        return 7  # Genitourinary
    elif 140 <= value < 240:
        return 8  # Neoplasms
    else:
        return 0  # Other

# Define categorization function for Level2 (example subset)
def categorize_level2(value):
    if 390 <= value < 399:
        return 1  # Rheumatic fever
    elif 401 <= value < 415:
        return 2  # Hypertension
    elif 415 <= value < 460:
        return 3  # Ischemic heart disease
    elif np.floor(value) == 785:
        return 4  # Heart failure
    else:
        return 0  # Other

# Apply categorization functions
for col in ['diag_1', 'diag_2', 'diag_3']:
    df[f'level1_{col}'] = df[f'level1_{col}'].apply(categorize_level1)
    df[f'level2_{col}'] = df[f'level2_{col}'].apply(categorize_level2)


In [89]:
df.head().T

,0,1,2,3,4
encounter_id,2278392,149190,64410,500364,16680
patient_nbr,8222157,55629189,86047875,82442376,42519267
race,Caucasian,Caucasian,AfricanAmerican,Caucasian,Caucasian
gender,0,0,0,1,1
age,1,2,3,4,5
admission_type_id,6,1,1,1,1
discharge_disposition_id,25,1,1,1,1
admission_source_id,1,7,7,7,7
time_in_hospital,1,3,2,2,1
num_lab_procedures,41,59,11,44,51


In [91]:
# drop individual diagnosis columns that have too granular disease information
# also drop level 2 categorization (which was not comparable with any reference)
# also drop level 1 secondary and tertiary diagnoses
df.drop(['diag_1', 'diag_2', 'diag_3', 'level2_diag_1', 'level1_diag_2', 'level2_diag_2', 'level1_diag_3',
         'level2_diag_3'], axis=1, inplace=True)

In [102]:
df.drop(['encounter_id','patient_nbr'],axis=1,inplace=True)

In [104]:
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fill NaN values in the 'race' column with 'Unknown' (or any preferred placeholder)
df['race'] = df['race'].fillna('Unknown')

# Apply label encoding to the 'race' column
df['race_encoded'] = label_encoder.fit_transform(df['race'])

# Display the mapping of labels to encoded values
race_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Race Mapping:", race_mapping)

# Preview the updated DataFrame
print(df[['race', 'race_encoded']].head())


Race Mapping: {'AfricanAmerican': np.int64(0), 'Asian': np.int64(1), 'Caucasian': np.int64(2), 'Hispanic': np.int64(3), 'Other': np.int64(4), 'Unknown': np.int64(5)}
              race  race_encoded
0        Caucasian             2
1        Caucasian             2
2  AfricanAmerican             0
3        Caucasian             2
4        Caucasian             2


In [105]:
# Replace the 'race' column with the encoded values
df['race'] = df['race_encoded']

# Drop the 'race_encoded' column if no longer needed
df.drop(columns=['race_encoded'], inplace=True)


   race gender  age  admission_type_id  discharge_disposition_id  \
0     2      0    1                  6                        25   
1     2      0    2                  1                         1   
2     0      0    3                  1                         1   
3     2      1    4                  1                         1   
4     2      1    5                  1                         1   

   admission_source_id  time_in_hospital  num_lab_procedures  num_procedures  \
0                    1                 1                  41               0   
1                    7                 3                  59               0   
2                    7                 2                  11               5   
3                    7                 2                  44               1   
4                    7                 1                  51               0   

   num_medications  ...  glipizide-metformin  glimepiride-pioglitazone  \
0                1  ...             

In [109]:
df['age'] = df['age'].astype('int64')
print(df.age.value_counts())
# convert age categories to mid-point values
age_dict = {1:5, 2:15, 3:25, 4:35, 5:45, 6:55, 7:65, 8:75, 9:85, 10:95}
df['age'] = df.age.map(age_dict)
print(df.age.value_counts())

age
8     18210
7     15960
6     12466
9     11589
5      6878
4      2699
10     1900
3      1127
2       535
1       154
Name: count, dtype: int64
age
75    18210
65    15960
55    12466
85    11589
45     6878
35     2699
95     1900
25     1127
15      535
5       154
Name: count, dtype: int64


In [116]:
df.drop(['max_glu_serum','A1Cresult'],axis=1,inplace=True)

In [117]:
df.to_csv('cleansed_data.csv', index=False)